<a href="https://colab.research.google.com/github/peasantxuan/prediction_fondation_model/blob/main/Fine_tuning_timesfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PREPARE ENVIRONMENT FOR TIMESFM
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.51.0'),
    ('timesfm', 'timesfm'),
    ('google.cloud.bigquery', 'google-cloud-bigquery')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)



project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

import os
os.environ['JAX_PLATFORMS'] = 'cpu'

import timesfm
import numpy as np
from google.cloud import bigquery
import pandas as pd
from matplotlib import pyplot as plt


bq = bigquery.Client(project = PROJECT_ID)


Are you sure you wish to set property [core/project] to statmike-mlops-349915?

Do you want to continue (Y/n)?  Y

Updated property [core/project].
Colab authorized to GCP
 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0].


In [ ]:
!pip install  torch transformers pandas numpy scikit-learn optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

torch_model.ckpt:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

In [ ]:
# Load & Preprocess the Dataset
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import requests
from io import StringIO

# Function to download data from Dropbox
# prompt: Get this csv file from dropbox: https://www.dropbox.com/scl/fi/9x3wgar4ky4o2exc7kosl/Dengue_Brazil_1980_2022.csv?rlkey=po2ig1g7msjknnb2p7o705egk&st=wo6ajcmp&dl=0. Give me a more simple one, less include new variable

import pandas as pd
import requests
from io import StringIO  # Make sure StringIO is imported


def get_dengue_data_from_dropbox():
    url = "https://www.dropbox.com/scl/fi/9x3wgar4ky4o2exc7kosl/Dengue_Brazil_1980_2022.csv?rlkey=po2ig1g7msjknnb2p7o705egk&st=gz3xp47x&dl=1"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise error if request fails
        df = pd.read_csv(StringIO(response.text), on_bad_lines='skip', delimiter=None)
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

dengue_data = get_dengue_data_from_dropbox()


def get_chickenpox_data_from_dropbox():
    url = "https://www.dropbox.com/scl/fi/rea28y2tizeohb8w88g1r/sentinelle_Chickenpox_France_1990_2025.csv?rlkey=rl0j8h528l4tgx7yxrjjmsth2&st=uteetgdz&dl=1"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise error if request fails
        df = pd.read_csv(StringIO(response.text), on_bad_lines='skip', delimiter=None)
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

chickenpox_data = get_chickenpox_data_from_dropbox()


def get_flu_data_from_dropbox():
    url = "https://www.dropbox.com/scl/fi/cf1j1vi7kfwsey3dxn5ks/sentinelle_Flu_France_2013_2025.csv?rlkey=f2ng6vw8yjx3w66noxk62ndqm&st=y2eexdfx&dl=1"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise error if request fails
        df = pd.read_csv(StringIO(response.text), on_bad_lines='skip', delimiter=None)
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

flu_data = get_flu_data_from_dropbox()




def get_ILI_data_from_dropbox():
    url = "https://www.dropbox.com/scl/fi/bn8q34o3f0jki4pp8cwkq/sentinelle_ILI_France_1984_2025.csv?rlkey=h57lvdkvvmpoifwprbl6nc4l2&st=s5ptg6t4&dl=1"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise error if request fails
        df = pd.read_csv(StringIO(response.text), on_bad_lines='skip', delimiter=None)
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

ILI_data = get_ILI_data_from_dropbox()



def get_RSV_data_from_dropbox():
    url = "https://www.dropbox.com/scl/fi/0cxaw7k9rvqwkwdtiwfi0/sentinelle_RSV_France_2013_2025.csv?rlkey=lh6i42gq7fm8xvry5si56y8y4&st=00bsm9ai&dl=1"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise error if request fails
        df = pd.read_csv(StringIO(response.text), on_bad_lines='skip', delimiter=None,sep =";")
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None
RSV_data = get_RSV_data_from_dropbox()

######fill the missing data######

import pandas as pd


def fill_missing_dates_with_previous_inc(df):
    # Sort the DataFrame by date
    df = df.sort_values(by='date')

    # Create a complete 7-day frequency date range between min and max dates
    full_date_range = pd.DataFrame({'date': pd.date_range(start=df['date'].min(),
                                                          end=df['date'].max(),
                                                          freq='7D')})

    # Merge the original DataFrame with the 7-day date range
    df_filled = full_date_range.merge(df, on='date', how='left')

    # Fill missing 'inc' values using the previous available value
    df_filled['inc'] = df_filled['inc'].fillna(method='ffill')

    # Ensure the column order is preserved: ['date', 'inc']
    df_filled = df_filled[['date', 'inc']]

    return df_filled




RSV_data = fill_missing_dates_with_previous_inc(RSV_data)
flu_data = fill_missing_dates_with_previous_inc(flu_data)
ILI_data = fill_missing_dates_with_previous_inc(ILI_data)
chickenpox_data = fill_missing_dates_with_previous_inc(chickenpox_data)
dengue_data = fill_missing_dates_with_previous_inc(dengue_data)
# Normalize the 'inc' column
#scaler = MinMaxScaler()
#ILI_data['inc'] = scaler.fit_transform(ILI_data[['inc']])

# Split into training and validation sets (80% train, 20% validation)
#train_data, val_data = train_test_split(ILI_data, test_size=0.2, shuffle=False)

#print(train_data.head(), val_data.head())



<ipython-input-7-cd7ea234fdda>:112: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled['inc'] = df_filled['inc'].fillna(method='ffill')
<ipython-input-7-cd7ea234fdda>:112: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled['inc'] = df_filled['inc'].fillna(method='ffill')
<ipython-input-7-cd7ea234fdda>:112: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled['inc'] = df_filled['inc'].fillna(method='ffill')
<ipython-input-7-cd7ea234fdda>:112: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled['inc'] = df_filled['inc'].fillna(method='ffill')
<ipython-input-7-cd7ea234fdda>:112: FutureWarning: Series.fillna with 'method' i

In [ ]:
#  Define Fine-Tuning Function

import pandas as pd
import numpy as np
import timesfm
import optuna
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Modified Prediction Function for Evaluation
def evaluate_timesfm(data, training_date_start, training_date_finish, predict_date_start, predict_date_finish,
                     context_len, batch_size):
    """Runs TimesFM forecasting and returns a loss metric (MSE) for hyperparameter tuning."""
    # Calculate the prediction length
    predict_length = ((pd.to_datetime(predict_date_finish) - pd.to_datetime(predict_date_start)).days) // 7 + 1

    # Ensure data is sorted by date and filtered for the relevant period
    data = data.sort_values(by='date').copy()

    # Filter training data
    training_data = data[(data['date'] >= training_date_start) & (data['date'] < training_date_finish)]

    # Initialize TimesFM model with given hyperparameters
    tfm = timesfm.TimesFm(
        hparams=timesfm.TimesFmHparams(
            backend="gpu",
            per_core_batch_size=batch_size,  # Tuned batch size
            horizon_len=predict_length,
            num_layers=50,  #  num_layers cannot be tunned
            use_positional_embedding=False,
            context_len=context_len,  # Tuned context length
        ),
        checkpoint=timesfm.TimesFmCheckpoint(
            huggingface_repo_id="google/timesfm-2.0-500m-pytorch"
        ),
    )

    # Forecast
    frequency_input = [1]  # Adjust if necessary
    point_forecast_FM, _ = tfm.forecast([training_data['inc'].values], freq=frequency_input)

    # Generate prediction dates
    last_date = training_data['date'].max()
    prediction_dates = pd.date_range(start=last_date, periods=predict_length, freq='7D')

    # Create DataFrame with predictions
    prediction_df = pd.DataFrame({
        'date': prediction_dates,
        'y': point_forecast_FM[0]
    })

    # Merge predictions with actual future values (for validation)
    merged_df = pd.merge(data, prediction_df, on='date', how='left')

    # Compute error metrics (MSE or MAE)
    actuals = merged_df.dropna(subset=['inc', 'y'])  # Ensure valid comparison
    mse = mean_squared_error(actuals['inc'], actuals['y'])
    mae = mean_absolute_error(actuals['inc'], actuals['y'])

    return mse  # Optuna will minimize this loss



#Define the Optuna Tuning Process




def objective(trial):
    """
    Objective function for Optuna hyperparameter optimization.
    """
    # Tune context_len, learning_rate, num_layers, and batch_size
    context_len = trial.suggest_categorical("context_len", [512, 1024, 2048])
    batch_size = trial.suggest_categorical("per_core_batch_size", [16, 32, 64])  # Tune batch size

    # Define training and validation periods
    training_date_start = "1984-10-29"
    training_date_finish = "2016-10-03"
    predict_date_start = "2016-10-03"
    predict_date_finish = "2019-05-27"


    # Evaluate the model using selected hyperparameters
    mse = evaluate_timesfm(
        ILI_data, training_date_start, training_date_finish, predict_date_start, predict_date_finish,
        context_len, batch_size
    )
    return mse  # Optuna will minimize this value

# Run Optuna hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)  # Reduce trials for computational efficiency

# Print best parameters
best_params = study.best_params
print("ILI Best Hyperparameters:", best_params)




[I 2025-03-05 14:44:25,933] A new study created in memory with name: no-name-e56010aa-6b45-4677-bfc2-cff21ff9feab


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:44:38,456] Trial 0 finished with value: 4321803166.757014 and parameters: {'context_len': 512, 'per_core_batch_size': 64}. Best is trial 0 with value: 4321803166.757014.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:44:50,837] Trial 1 finished with value: 4033349588.9472303 and parameters: {'context_len': 1024, 'per_core_batch_size': 32}. Best is trial 1 with value: 4033349588.9472303.


...

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:48:12,248] Trial 13 finished with value: 4033349588.9472303 and parameters: {'context_len': 1024, 'per_core_batch_size': 32}. Best is trial 1 with value: 4033349588.9472303.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:48:24,464] Trial 14 finished with value: 4033349588.9472303 and parameters: {'context_len': 1024, 'per_core_batch_size': 32}. Best is trial 1 with value: 4033349588.9472303.


ILI Best Hyperparameters: {'context_len': 1024, 'per_core_batch_size': 32}


In [ ]:
#Best Hyperparameters for ILI: {'context_len': 1024, 'learning_rate': 6.423499993725044e-05, 'per_core_batch_size': 64}



In [ ]:

def objective(trial):
    """
    Objective function for Optuna hyperparameter optimization.
    """
    # Tune context_len, learning_rate, num_layers, and batch_size
    context_len = trial.suggest_categorical("context_len", [512, 1024, 2048])
    batch_size = trial.suggest_categorical("per_core_batch_size", [16, 32, 64])  # Tune batch size

    # Define training and validation periods
    training_date_start = "2013-12-30"
    training_date_finish = "2016-10-03"
    predict_date_start = "2015-10-03"
    predict_date_finish = "2016-10-03"


    # Evaluate the model using selected hyperparameters
    mse = evaluate_timesfm(
        flu_data, training_date_start, training_date_finish, predict_date_start, predict_date_finish,
        context_len, batch_size
    )
    return mse  # Optuna will minimize this value

# Run Optuna hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)  # Reduce trials for computational efficiency

# Print best parameters
best_params = study.best_params
print("Flu Best Hyperparameters:", best_params)

[I 2025-03-05 14:56:21,887] A new study created in memory with name: no-name-b17cb3dc-f7cc-452e-aa78-e08c9b7cac5a


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:56:27,660] Trial 0 finished with value: 48762026.86125678 and parameters: {'context_len': 512, 'per_core_batch_size': 32}. Best is trial 0 with value: 48762026.86125678.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:56:32,475] Trial 1 finished with value: 48762026.86125678 and parameters: {'context_len': 512, 'per_core_batch_size': 16}. Best is trial 0 with value: 48762026.86125678.


...

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:58:11,943] Trial 13 finished with value: 48762026.86125678 and parameters: {'context_len': 512, 'per_core_batch_size': 32}. Best is trial 0 with value: 48762026.86125678.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:58:19,747] Trial 14 finished with value: 48762026.86125678 and parameters: {'context_len': 512, 'per_core_batch_size': 64}. Best is trial 0 with value: 48762026.86125678.


Flu Best Hyperparameters: {'context_len': 512, 'per_core_batch_size': 32}


In [ ]:

def objective(trial):
    """
    Objective function for Optuna hyperparameter optimization.
    """
    # Tune context_len, learning_rate, num_layers, and batch_size
    context_len = trial.suggest_categorical("context_len", [512, 1024, 2048])
    batch_size = trial.suggest_categorical("per_core_batch_size", [16, 32, 64])  # Tune batch size

    # Define training and validation periods
    training_date_start = "2014-10-06"
    training_date_finish = "2016-10-03"
    predict_date_start = "2015-10-03"
    predict_date_finish = "2016-10-03"


    # Evaluate the model using selected hyperparameters
    mse = evaluate_timesfm(
        RSV_data, training_date_start, training_date_finish, predict_date_start, predict_date_finish,
        context_len, batch_size
    )
    return mse  # Optuna will minimize this value

# Run Optuna hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)  # Reduce trials for computational efficiency

# Print best parameters
best_params = study.best_params
print("RSV Best Hyperparameters:", best_params)

[I 2025-03-05 14:51:33,651] A new study created in memory with name: no-name-9419fee4-4b5d-4347-bdb0-c8132bb20eaf


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:51:46,184] Trial 0 finished with value: 26962253.799513035 and parameters: {'context_len': 2048, 'per_core_batch_size': 32}. Best is trial 0 with value: 26962253.799513035.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:51:54,016] Trial 1 finished with value: 26962253.799513035 and parameters: {'context_len': 512, 'per_core_batch_size': 64}. Best is trial 0 with value: 26962253.799513035.


...

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:54:19,651] Trial 13 finished with value: 26962253.799513035 and parameters: {'context_len': 512, 'per_core_batch_size': 32}. Best is trial 0 with value: 26962253.799513035.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:54:25,463] Trial 14 finished with value: 26962253.799513035 and parameters: {'context_len': 1024, 'per_core_batch_size': 16}. Best is trial 0 with value: 26962253.799513035.


RSV Best Hyperparameters: {'context_len': 2048, 'per_core_batch_size': 32}


In [ ]:

def objective(trial):
    """
    Objective function for Optuna hyperparameter optimization.
    """
    # Tune context_len, learning_rate, num_layers, and batch_size
    context_len = trial.suggest_categorical("context_len", [512, 1024, 2048])
    batch_size = trial.suggest_categorical("per_core_batch_size", [16, 32, 64])  # Tune batch size

    # Define training and validation periods
    training_date_start = "1990-12-03"
    training_date_finish = "2016-10-03"
    predict_date_start = "2015-10-03"
    predict_date_finish = "2016-10-03"


    # Evaluate the model using selected hyperparameters
    mse = evaluate_timesfm(
        chickenpox_data, training_date_start, training_date_finish, predict_date_start, predict_date_finish,
        context_len, batch_size
    )
    return mse  # Optuna will minimize this value

# Run Optuna hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)  # Reduce trials for computational efficiency

# Print best parameters
best_params = study.best_params
print("chickenpox Best Hyperparameters:", best_params)

[I 2025-03-05 14:48:24,481] A new study created in memory with name: no-name-7fcbbaf1-bb09-4df6-a842-e41c316cbd09


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:48:36,911] Trial 0 finished with value: 13208048.051793203 and parameters: {'context_len': 2048, 'per_core_batch_size': 32}. Best is trial 0 with value: 13208048.051793203.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:48:44,812] Trial 1 finished with value: 20016233.663802758 and parameters: {'context_len': 512, 'per_core_batch_size': 64}. Best is trial 0 with value: 13208048.051793203.


...

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:51:12,401] Trial 13 finished with value: 13208048.051793203 and parameters: {'context_len': 2048, 'per_core_batch_size': 32}. Best is trial 0 with value: 13208048.051793203.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:51:33,642] Trial 14 finished with value: 13208048.051793203 and parameters: {'context_len': 2048, 'per_core_batch_size': 64}. Best is trial 0 with value: 13208048.051793203.


chickenpox Best Hyperparameters: {'context_len': 2048, 'per_core_batch_size': 32}


In [ ]:

def objective(trial):
    """
    Objective function for Optuna hyperparameter optimization.
    """
    # Tune context_len, learning_rate, num_layers, and batch_size
    context_len = trial.suggest_categorical("context_len", [512, 1024, 2048])
    batch_size = trial.suggest_categorical("per_core_batch_size", [16, 32, 64])  # Tune batch size

    # Define training and validation periods
    training_date_start = "2015-01-04"
    training_date_finish = "2016-10-03"
    predict_date_start = "2015-10-03"
    predict_date_finish = "2016-10-03"


    # Evaluate the model using selected hyperparameters
    mse = evaluate_timesfm(
        dengue_data, training_date_start, training_date_finish, predict_date_start, predict_date_finish,
        context_len, batch_size
    )
    return mse  # Optuna will minimize this value

# Run Optuna hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)  # Reduce trials for computational efficiency

# Print best parameters
best_params = study.best_params
print("dengue Best Hyperparameters:", best_params)

[I 2025-03-05 14:54:25,475] A new study created in memory with name: no-name-94544ab2-0651-4f33-87de-22e80a76e2f5


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:54:31,406] Trial 0 finished with value: 726493691.8218842 and parameters: {'context_len': 512, 'per_core_batch_size': 32}. Best is trial 0 with value: 726493691.8218842.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:54:36,217] Trial 1 finished with value: 726493691.8218842 and parameters: {'context_len': 512, 'per_core_batch_size': 16}. Best is trial 0 with value: 726493691.8218842.


...

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:56:13,738] Trial 13 finished with value: 726493691.8218842 and parameters: {'context_len': 512, 'per_core_batch_size': 16}. Best is trial 0 with value: 726493691.8218842.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-03-05 14:56:21,707] Trial 14 finished with value: 726493691.8218842 and parameters: {'context_len': 512, 'per_core_batch_size': 64}. Best is trial 0 with value: 726493691.8218842.


dengue Best Hyperparameters: {'context_len': 512, 'per_core_batch_size': 32}
